In [ ]:
import sqlite3

# Connect to an existing database file
conn = sqlite3.connect('Chinook.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute a SELECT query
cursor.execute('SELECT * FROM Artist LIMIT 10')

# Fetch the query results
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)

# Close the cursor and database connection
cursor.close()
conn.close()

(1, 'AC/DC')
(2, 'Accept')
(3, 'Aerosmith')
(4, 'Alanis Morissette')
(5, 'Alice In Chains')
(6, 'Antônio Carlos Jobim')
(7, 'Apocalyptica')
(8, 'Audioslave')
(9, 'BackBeat')
(10, 'Billy Cobham')


In [ ]:
pip install langchain openai

In [ ]:
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain import SQLDatabaseChain
from langchain.chains import SQLDatabaseSequentialChain
from urllib.parse import quote
from sqlalchemy import create_engine, select, Table, MetaData, Column, String
import os


os.environ['OPENAI_API_KEY'] = "your_key"




In [ ]:
db_chain("Find the top 5 customers who have spent the most on Rock music, listing their total spend, and the details of the employee who supports them.")



> Entering new SQLDatabaseChain chain...
Find the top 5 customers who have spent the most on Rock music, listing their total spend, and the details of the employee who supports them.
SQLQuery:WITH CustomerRockSpend AS (
    SELECT c.CustomerId, c.FirstName || ' ' || c.LastName AS CustomerName, SUM(il.UnitPrice) AS TotalSpend, c.SupportRepId
    FROM Customer c
    JOIN Invoice i ON c.CustomerId = i.CustomerId
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    JOIN Track t ON il.TrackId = t.TrackId
    JOIN Genre g ON t.GenreId = g.GenreId
    WHERE g.Name = 'Rock'
    GROUP BY c.CustomerId
    ORDER BY TotalSpend DESC
    LIMIT 5
)
SELECT crs.CustomerName, crs.TotalSpend, e.FirstName || ' ' || e.LastName AS EmployeeName
FROM CustomerRockSpend crs
JOIN Employee e ON crs.SupportRepId = e.EmployeeId;
SQLResult: [('Eduardo Martins', 28.709999999999983, 'Margaret Park'), ('Robert Brown', 24.74999999999999, 'Jane Peacock'), ('Mark Taylor', 21.779999999999994, 'Margaret Park'), ('Enr

{'query': 'Find the top 5 customers who have spent the most on Rock music, listing their total spend, and the details of the employee who supports them.',
 'result': 'The top 5 customers who have spent the most on Rock music, along with their total spend and the details of the employee who supports them, are as follows:\n\n1. Customer: Eduardo Martins, Total Spend: $28.71, Employee: Margaret Park\n2. Customer: Robert Brown, Total Spend: $24.75, Employee: Jane Peacock\n3. Customer: Mark Taylor, Total Spend: $21.78, Employee: Margaret Park\n4. Customer: Enrique Muñoz, Total Spend: $21.78, Employee: Steve Johnson\n5. Customer: Stanisław Wójcik, Total Spend: $21.78, Employee: Margaret Park'}

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

In [ ]:
from langchain.prompts.prompt import PromptTemplate


_DEFAULT_TEMPLATE = """You are a SQL expert. Given an input question, first understand the question, examine all the table names in the database, then create a syntactically correct {dialect} query to run, then analyze the results of the query and return the answer.
Do not use aliases that are reserved keywords in SQL. Pay close attention to the database schema. Use the following format:
Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

{table_info}
Below are all the tables and their respective joins between the primary key and foreign key in the database:
'Album' table's 'ArtistId' with 'Artist' table's 'ArtistId'
'Customer' table's 'SupportRepId' with 'Employee' table's 'EmployeeId'
'Employee' table's 'ReportsTo' with 'Employee' table's 'EmployeeId'
'Invoice' table's 'CustomerId' with 'Customer' table's 'CustomerId'
'InvoiceLine' table's 'TrackId' with 'Track' table's 'TrackId'
'InvoiceLine' table's 'InvoiceId' with 'Invoice' table's 'InvoiceId'
'PlaylistTrack' table's 'TrackId' with 'Track' table's 'TrackId'
'PlaylistTrack' table's 'PlaylistId' with 'Playlist' table's 'PlaylistId'
'Track' table's 'MediaTypeId' with 'MediaType' table's 'MediaTypeId'
'Track' table's 'GenreId' with 'Genre' table's 'GenreId'
'Track' table's 'AlbumId' with 'Album' table's 'AlbumId'
Differentiate between audio and video tracks based on the 'MediaType' table.

Below are a few examples of Expert level queries:

Query#1:
WITH AlbumSales AS (
    SELECT
        Album.AlbumId,
        MIN(Invoice.InvoiceDate) AS FirstSale,
        MAX(Invoice.InvoiceDate) AS LastSale,
        COUNT(*) AS Sales
    FROM
        InvoiceLine
    JOIN
        Invoice ON InvoiceLine.InvoiceId = Invoice.InvoiceId
    JOIN
        Track ON InvoiceLine.TrackId = Track.TrackId
    JOIN
        Album ON Track.AlbumId = Album.AlbumId
    GROUP BY
        Album.AlbumId
    HAVING
        Sales > 1 -- Consider only albums with more than one sale
),
SalesIntervals AS (
    SELECT
        AlbumId,
        Sales,
        (JulianDay(LastSale) - JulianDay(FirstSale)) / (Sales - 1) AS AvgDaysBetweenSales
    FROM
        AlbumSales
)
SELECT
    Album.Title AS Album,
    Artist.Name AS Artist,
    SalesIntervals.Sales,
    SalesIntervals.AvgDaysBetweenSales
FROM
    SalesIntervals
JOIN
    Album ON SalesIntervals.AlbumId = Album.AlbumId
JOIN
    Artist ON Album.ArtistId = Artist.ArtistId
ORDER BY
    AvgDaysBetweenSales ASC;

Query#2:
WITH CustomerAlbumCounts AS (
    SELECT c.CustomerId, a.AlbumId, COUNT(*) as Count
    FROM Invoice i
    JOIN Customer c ON i.CustomerId = c.CustomerId
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    JOIN Track t ON il.TrackId = t.TrackId
    JOIN Album a ON t.AlbumId = a.AlbumId
    GROUP BY c.CustomerId, a.AlbumId
)

SELECT c1.CustomerId as CustomerId1, c1.FirstName || ' ' || c1.LastName as CustomerName1,
       c2.CustomerId as CustomerId2, c2.FirstName || ' ' || c2.LastName as CustomerName2,
       a.Title as Album, cac1.Count + cac2.Count as TotalCount
FROM CustomerAlbumCounts cac1
JOIN CustomerAlbumCounts cac2 ON cac1.AlbumId = cac2.AlbumId AND cac1.CustomerId < cac2.CustomerId
JOIN Customer c1 ON cac1.CustomerId = c1.CustomerId
JOIN Customer c2 ON cac2.CustomerId = c2.CustomerId
JOIN Album a ON cac1.AlbumId = a.AlbumId
ORDER BY TotalCount DESC;

Query#3:
WITH ArtistPurchaseDates AS (
    SELECT i.InvoiceDate, a.ArtistId, COUNT(*) as Count
    FROM Invoice i
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    JOIN Track t ON il.TrackId = t.TrackId
    JOIN Album a ON t.AlbumId = a.AlbumId
    GROUP BY i.InvoiceDate, a.ArtistId
)
SELECT ar1.Name as Artist1, ar2.Name as Artist2, SUM(apd1.Count + apd2.Count) as TotalCount
FROM ArtistPurchaseDates apd1
JOIN ArtistPurchaseDates apd2 ON apd1.InvoiceDate = apd2.InvoiceDate AND apd1.ArtistId < apd2.ArtistId
JOIN Artist ar1 ON apd1.ArtistId = ar1.ArtistId
JOIN Artist ar2 ON apd2.ArtistId = ar2.ArtistId
GROUP BY apd1.ArtistId, apd2.ArtistId
ORDER BY TotalCount DESC;

Query#4:
SELECT Artist.Name, Track.Name, SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS Revenue
FROM Track
JOIN Album ON Track.AlbumId=Album.AlbumId
JOIN Artist ON Album.ArtistId = Artist.ArtistId and track.Composer=Artist.Name
JOIN InvoiceLine on Track.TrackId=InvoiceLine.TrackId
group by Artist.Name, Track.Name
order by Revenue DESC;

Question: {input}"""""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

db = SQLDatabase.from_uri("sqlite:///./Chinook.db")

toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0, model_name='gpt-3.5-turbo-16k-0613'))
db_chain = SQLDatabaseChain(llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-16k-0613'), database=db, memory=memory, verbose=True, prompt=PROMPT)

agent_executor = create_sql_agent(
     llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k-0613"),
     toolkit=toolkit,
     verbose=True,
     sql_chain=db_chain,
     agent_type=AgentType.OPENAI_FUNCTIONS,
     memory=memory)

In [ ]:
result = db_chain("For Rock, what are the top 3 tracks that have the longest duration?")

tracks = result['result'].split(',')
for track in tracks:
    print(track.strip())




> Entering new SQLDatabaseChain chain...
For Rock, what are the top 3 tracks that have the longest duration?
SQLQuery:WITH RockTracks AS (
    SELECT
        Track.Name,
        Track.Milliseconds
    FROM
        Track
    JOIN
        Genre ON Track.GenreId = Genre.GenreId
    WHERE
        Genre.Name = 'Rock'
)
SELECT
    Name,
    Milliseconds
FROM
    RockTracks
ORDER BY
    Milliseconds DESC
LIMIT 3;
SQLResult: [('Dazed And Confused', 1612329), ("Space Truckin'", 1196094), ('Dazed And Confused', 1116734)]
Answer:The top 3 tracks in the Rock genre with the longest duration are:
1. "Dazed And Confused" with a duration of 1612329 milliseconds
2. "Space Truckin'" with a duration of 1196094 milliseconds
3. "Dazed And Confused" with a duration of 1116734 milliseconds
> Finished chain.
The top 3 tracks in the Rock genre with the longest duration are:
1. "Dazed And Confused" with a duration of 1612329 milliseconds
2. "Space Truckin'" with a duration of 1196094 milliseconds
3. "Dazed An

In [ ]:
chain.run("For each customer in Argentina, what was their first invoice and how much was it?")



> Entering new SQLDatabaseSequentialChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['Customer', 'Invoice', 'InvoiceLine']

> Entering new SQLDatabaseChain chain...
For each customer in Argentina, what was their first invoice and how much was it?
SQLQuery:SELECT c.FirstName, c.LastName, MIN(i.InvoiceDate) AS FirstInvoiceDate, MIN(i.Total) AS FirstInvoiceTotal
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
WHERE c.Country = 'Argentina'
GROUP BY c.CustomerId
SQLResult: [('Diego', 'Gutiérrez', '2010-06-12 00:00:00', 0.99)]
Answer:For each customer in Argentina, their first invoice was on June 12, 2010, and the total amount was $0.99.
> Finished chain.

> Finished chain.


'For each customer in Argentina, their first invoice was on June 12, 2010, and the total amount was $0.99.'

In [ ]:
result= db_chain("Show me the Top 10 customers based on cumulative revenue over time, ranked by the total revenue.")
tracks = result['result'].split(',')
for track in tracks:
    print(track.strip())



> Entering new SQLDatabaseChain chain...
Show me the Top 10 customers based on cumulative revenue over time, ranked by the total revenue.
SQLQuery:WITH CustomerRevenue AS (
    SELECT
        c.CustomerId,
        c.FirstName || ' ' || c.LastName AS CustomerName,
        SUM(i.Total) AS CumulativeRevenue
    FROM
        Customer c
    JOIN
        Invoice i ON c.CustomerId = i.CustomerId
    GROUP BY
        c.CustomerId, c.FirstName, c.LastName
)
SELECT
    CustomerName,
    CumulativeRevenue
FROM
    CustomerRevenue
ORDER BY
    CumulativeRevenue DESC
LIMIT 10;
SQLResult: [('Helena Holý', 49.620000000000005), ('Richard Cunningham', 47.620000000000005), ('Luis Rojas', 46.62), ('Ladislav Kovács', 45.62), ("Hugh O'Reilly", 45.62), ('Julia Barnett', 43.620000000000005), ('Frank Ralston', 43.62), ('Fynn Zimmermann', 43.62), ('Astrid Gruber', 42.62), ('Victor Stevens', 42.62)]
Answer:The top 10 customers based on cumulative revenue over time, ranked by the total revenue are:

1. Helena 

In [ ]:
db_chain("Did customers in Argentina make us more money than Brazil?")



> Entering new SQLDatabaseChain chain...
Did customers in Argentina make us more money than Brazil?
SQLQuery:WITH ArgentinaRevenue AS (
    SELECT SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS ArgentinaRevenue
    FROM Invoice
    JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
    JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
    JOIN Track ON InvoiceLine.TrackId = Track.TrackId
    JOIN Album ON Track.AlbumId = Album.AlbumId
    JOIN Artist ON Album.ArtistId = Artist.ArtistId
    WHERE Customer.Country = 'Argentina'
),
BrazilRevenue AS (
    SELECT SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS BrazilRevenue
    FROM Invoice
    JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
    JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
    JOIN Track ON InvoiceLine.TrackId = Track.TrackId
    JOIN Album ON Track.AlbumId = Album.AlbumId
    JOIN Artist ON Album.ArtistId = Artist.ArtistId
    WHERE Customer.Country = 'Brazil'
)
SE

{'query': 'Did customers in Argentina make us more money than Brazil?',
 'history': 'Human: List all distinct playlists and whether they are audio or video.\nAI: There are 15 distinct playlists in the database. Some playlists are labeled as "Audio" while others are labeled as "Video". The "Music" playlist contains both audio and video tracks. The "TV Shows" playlist contains only video tracks. The rest of the playlists are labeled as "Audio".\nHuman: Show me total revenue by country.\nAI: Total revenue by country:\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n11. Ireland: $45.62\n12. Hungary: $45.62\n13. Austria: $42.62\n14. Finland: $41.62\n15. Netherlands: $40.62\n16. Norway: $39.62\n17. Sweden: $38.62\n18. Poland: $37.62\n19. Italy: $37.62\n20. Denmark: $37.62\n21. Australia: $37.62\n22. Argentina: $37.62\n23. Spain: $3

In [ ]:
db_chain("How many unique artists represent each genre?")



> Entering new SQLDatabaseChain chain...
How many unique artists represent each genre?
SQLQuery:WITH UniqueArtists AS (
    SELECT DISTINCT Genre.Name, Artist.Name
    FROM Genre
    JOIN Track ON Genre.GenreId = Track.GenreId
    JOIN Album ON Track.AlbumId = Album.AlbumId
    JOIN Artist ON Album.ArtistId = Artist.ArtistId
)
SELECT Name, COUNT(*) AS UniqueArtistsCount
FROM UniqueArtists
GROUP BY Name;
SQLResult: [('Alternative', 5), ('Alternative & Punk', 16), ('Blues', 5), ('Bossa Nova', 1), ('Classical', 66), ('Comedy', 1), ('Drama', 2), ('Easy Listening', 1), ('Electronica/Dance', 3), ('Heavy Metal', 1), ('Hip Hop/Rap', 2), ('Jazz', 10), ('Latin', 28), ('Metal', 14), ('Opera', 1), ('Pop', 3), ('R&B/Soul', 4), ('Reggae', 3), ('Rock', 51), ('Rock And Roll', 1), ('Sci Fi & Fantasy', 2), ('Science Fiction', 1), ('Soundtrack', 4), ('TV Shows', 5), ('World', 3)]
Answer:There are 24 unique genres in the database.
> Finished chain.


{'query': 'How many unique artists represent each genre?',
 'result': 'There are 24 unique genres in the database.'}

In [ ]:
result = db_chain("List all distinct playlists and whether they are audio or video.")
tracks = result['result'].split(',')
for track in tracks:
    print(track.strip())



> Entering new SQLDatabaseChain chain...
List all distinct playlists and whether they are audio or video.
SQLQuery:SELECT 
    Playlist.Name AS Playlist,
    CASE
        WHEN MediaType.Name LIKE '%audio%' THEN 'Audio'
        WHEN MediaType.Name LIKE '%video%' THEN 'Video'
        ELSE 'Unknown'
    END AS Type
FROM 
    Playlist
JOIN 
    PlaylistTrack ON Playlist.PlaylistId = PlaylistTrack.PlaylistId
JOIN 
    Track ON PlaylistTrack.TrackId = Track.TrackId
JOIN 
    MediaType ON Track.MediaTypeId = MediaType.MediaTypeId
GROUP BY 
    Playlist.PlaylistId, Type;
SQLResult: [('Music', 'Audio'), ('Music', 'Video'), ('TV Shows', 'Video'), ('90’s Music', 'Audio'), ('Music', 'Audio'), ('Music', 'Video'), ('Music Videos', 'Video'), ('TV Shows', 'Video'), ('Brazilian Music', 'Audio'), ('Classical', 'Audio'), ('Classical 101 - Deep Cuts', 'Audio'), ('Classical 101 - Next Steps', 'Audio'), ('Classical 101 - The Basics', 'Audio'), ('Grunge', 'Audio'), ('Heavy Metal Classic', 'Audio'), ('On-Th

In [ ]:
db_chain("How many unique artists represent each genre?")



> Entering new SQLDatabaseChain chain...
How many unique artists represent each genre?
SQLQuery:WITH UniqueArtists AS (
    SELECT Genre.Name AS Genre, COUNT(DISTINCT Artist.ArtistId) AS UniqueArtists
    FROM Genre
    JOIN Track ON Genre.GenreId = Track.GenreId
    JOIN Album ON Track.AlbumId = Album.AlbumId
    JOIN Artist ON Album.ArtistId = Artist.ArtistId
    GROUP BY Genre.Name
)
SELECT Genre, UniqueArtists
FROM UniqueArtists
ORDER BY UniqueArtists DESC;
SQLResult: [('Classical', 66), ('Rock', 51), ('Latin', 28), ('Alternative & Punk', 16), ('Metal', 14), ('Jazz', 10), ('Alternative', 5), ('Blues', 5), ('TV Shows', 5), ('R&B/Soul', 4), ('Soundtrack', 4), ('Electronica/Dance', 3), ('Pop', 3), ('Reggae', 3), ('World', 3), ('Drama', 2), ('Hip Hop/Rap', 2), ('Sci Fi & Fantasy', 2), ('Bossa Nova', 1), ('Comedy', 1), ('Easy Listening', 1), ('Heavy Metal', 1), ('Opera', 1), ('Rock And Roll', 1), ('Science Fiction', 1)]
Answer:There are 66 unique artists representing the Classical gen

{'query': 'How many unique artists represent each genre?',
 'result': 'There are 66 unique artists representing the Classical genre, 51 unique artists representing the Rock genre, 28 unique artists representing the Latin genre, and so on.'}

In [ ]:
db_chain("Based on the birth months of the employees, display how much revenue was recorded for each birth month.")



> Entering new SQLDatabaseChain chain...
Based on the birth months of the employees, display how much revenue was recorded for each birth month.
SQLQuery:WITH EmployeeBirthMonths AS (
    SELECT
        EmployeeId,
        STRFTIME('%m', BirthDate) AS BirthMonth
    FROM
        Employee
),
EmployeeRevenue AS (
    SELECT
        EmployeeBirthMonths.BirthMonth,
        SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS Revenue
    FROM
        EmployeeBirthMonths
    JOIN
        Employee ON EmployeeBirthMonths.EmployeeId = Employee.EmployeeId
    JOIN
        Customer ON Employee.EmployeeId = Customer.SupportRepId
    JOIN
        Invoice ON Customer.CustomerId = Invoice.CustomerId
    JOIN
        InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
    GROUP BY
        EmployeeBirthMonths.BirthMonth
)
SELECT
    EmployeeBirthMonths.BirthMonth,
    EmployeeRevenue.Revenue
FROM
    EmployeeBirthMonths
JOIN
    EmployeeRevenue ON EmployeeBirthMonths.BirthMonth = EmployeeRevenue

{'query': 'Based on the birth months of the employees, display how much revenue was recorded for each birth month.',
 'history': 'Human: List all distinct playlists and whether they are audio or video.\nAI: There are 15 distinct playlists in the database. Some playlists are labeled as "Audio" while others are labeled as "Video". The "Music" playlist contains both audio and video tracks. The "TV Shows" playlist contains only video tracks. The rest of the playlists are labeled as "Audio".\nHuman: Show me total revenue by country.\nAI: Total revenue by country:\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n11. Ireland: $45.62\n12. Hungary: $45.62\n13. Austria: $42.62\n14. Finland: $41.62\n15. Netherlands: $40.62\n16. Norway: $39.62\n17. Sweden: $38.62\n18. Poland: $37.62\n19. Italy: $37.62\n20. Denmark: $37.62\n21. Australia:

In [ ]:
db_chain("Rank Top 10 customer names and their country based on total revenue.")



> Entering new SQLDatabaseChain chain...
Rank Top 10 customer names and their country based on total revenue.
SQLQuery:WITH CustomerRevenue AS (
    SELECT
        c.CustomerId,
        c.FirstName || ' ' || c.LastName AS CustomerName,
        c.Country,
        SUM(i.Total) AS TotalRevenue
    FROM
        Customer c
    JOIN
        Invoice i ON c.CustomerId = i.CustomerId
    GROUP BY
        c.CustomerId
)
SELECT
    CustomerName,
    Country,
    TotalRevenue
FROM
    CustomerRevenue
ORDER BY
    TotalRevenue DESC
LIMIT 10;
SQLResult: [('Helena Holý', 'Czech Republic', 49.620000000000005), ('Richard Cunningham', 'USA', 47.620000000000005), ('Luis Rojas', 'Chile', 46.62), ('Ladislav Kovács', 'Hungary', 45.62), ("Hugh O'Reilly", 'Ireland', 45.62), ('Julia Barnett', 'USA', 43.620000000000005), ('Frank Ralston', 'USA', 43.62), ('Fynn Zimmermann', 'Germany', 43.62), ('Astrid Gruber', 'Austria', 42.62), ('Victor Stevens', 'USA', 42.62)]
Answer:The top 10 customer names and their count

{'query': 'Rank Top 10 customer names and their country based on total revenue.',
 'history': 'Human: List all distinct playlists and whether they are audio or video.\nAI: There are 15 distinct playlists in the database. Some playlists are labeled as "Audio" while others are labeled as "Video". The "Music" playlist contains both audio and video tracks. The "TV Shows" playlist contains only video tracks. The rest of the playlists are labeled as "Audio".\nHuman: Show me total revenue by country.\nAI: Total revenue by country:\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n11. Ireland: $45.62\n12. Hungary: $45.62\n13. Austria: $42.62\n14. Finland: $41.62\n15. Netherlands: $40.62\n16. Norway: $39.62\n17. Sweden: $38.62\n18. Poland: $37.62\n19. Italy: $37.62\n20. Denmark: $37.62\n21. Australia: $37.62\n22. Argentina: $37.62\n23.

In [ ]:
db_chain("How many distinct artists compose their own tracks and how many don't?")



> Entering new SQLDatabaseChain chain...
How many distinct artists compose their own tracks and how many don't?
SQLQuery:WITH SelfComposed AS (
    SELECT Artist.ArtistId, COUNT(DISTINCT Track.TrackId) AS SelfComposedTracks
    FROM Artist
    JOIN Album ON Artist.ArtistId = Album.ArtistId
    JOIN Track ON Album.AlbumId = Track.AlbumId
    WHERE Track.Composer = Artist.Name
    GROUP BY Artist.ArtistId
),
TotalArtists AS (
    SELECT COUNT(*) AS TotalArtists
    FROM Artist
)
SELECT 
    (SELECT COUNT(*) FROM SelfComposed) AS ArtistsWithSelfComposedTracks,
    TotalArtists.TotalArtists - (SELECT COUNT(*) FROM SelfComposed) AS ArtistsWithoutSelfComposedTracks
FROM TotalArtists;
SQLResult: [(41, 234)]
Answer:There are 41 artists who compose their own tracks, and 234 artists who do not compose their own tracks.
> Finished chain.


{'query': "How many distinct artists compose their own tracks and how many don't?",
 'history': 'Human: List all distinct playlists and whether they are audio or video.\nAI: There are 15 distinct playlists in the database. Some playlists are labeled as "Audio" while others are labeled as "Video". The "Music" playlist contains both audio and video tracks. The "TV Shows" playlist contains only video tracks. The rest of the playlists are labeled as "Audio".\nHuman: Show me total revenue by country.\nAI: Total revenue by country:\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n11. Ireland: $45.62\n12. Hungary: $45.62\n13. Austria: $42.62\n14. Finland: $41.62\n15. Netherlands: $40.62\n16. Norway: $39.62\n17. Sweden: $38.62\n18. Poland: $37.62\n19. Italy: $37.62\n20. Denmark: $37.62\n21. Australia: $37.62\n22. Argentina: $37.62\n2

In [ ]:
db_chain("Which track has the highest total revenue where the artist and composer are same?")



> Entering new SQLDatabaseChain chain...
Which track has the highest total revenue where the artist and composer are same?
SQLQuery:SELECT Artist.Name, Track.Name, SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS Revenue
FROM Track 
JOIN Album ON Track.AlbumId=Album.AlbumId
JOIN Artist ON Album.ArtistId = Artist.ArtistId and track.Composer=Artist.Name
JOIN InvoiceLine on Track.TrackId=InvoiceLine.TrackId
GROUP BY Artist.Name, Track.Name
ORDER BY Revenue DESC
LIMIT 1;
SQLResult: [('The Tea Party', 'Release', 2.9699999999999998)]
Answer:The track "Release" by the artist "The Tea Party" has the highest total revenue where the artist and composer are the same.
> Finished chain.


{'query': 'Which track has the highest total revenue where the artist and composer are same?',
 'history': 'Human: List all distinct playlists and whether they are audio or video.\nAI: There are 15 distinct playlists in the database. Some playlists are labeled as "Audio" while others are labeled as "Video". The "Music" playlist contains both audio and video tracks. The "TV Shows" playlist contains only video tracks. The rest of the playlists are labeled as "Audio".\nHuman: Show me total revenue by country.\nAI: Total revenue by country:\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n11. Ireland: $45.62\n12. Hungary: $45.62\n13. Austria: $42.62\n14. Finland: $41.62\n15. Netherlands: $40.62\n16. Norway: $39.62\n17. Sweden: $38.62\n18. Poland: $37.62\n19. Italy: $37.62\n20. Denmark: $37.62\n21. Australia: $37.62\n22. Argentina

In [ ]:
db_chain("Top 10 pair of tracks that were purchased by the same customer. Rank based on the total number of customers who purchased.")



> Entering new SQLDatabaseChain chain...
Top 10 pair of tracks that were purchased by the same customer. Rank based on the total number of customers who purchased.
SQLQuery:WITH TrackPairs AS (
    SELECT
        il1.TrackId AS TrackId1,
        il2.TrackId AS TrackId2,
        COUNT(DISTINCT i.CustomerId) AS CustomerCount
    FROM
        InvoiceLine il1
    JOIN
        InvoiceLine il2 ON il1.InvoiceId = il2.InvoiceId AND il1.TrackId < il2.TrackId
    JOIN
        Invoice i ON il1.InvoiceId = i.InvoiceId
    GROUP BY
        il1.TrackId,
        il2.TrackId
)
SELECT
    t1.Name AS Track1,
    t2.Name AS Track2,
    tp.CustomerCount
FROM
    TrackPairs tp
JOIN
    Track t1 ON tp.TrackId1 = t1.TrackId
JOIN
    Track t2 ON tp.TrackId2 = t2.TrackId
ORDER BY
    tp.CustomerCount DESC
LIMIT 10;
SQLResult: [('Snowballed', 'String Quartet No. 12 in C Minor, D. 703 "Quartettsatz": II. Andante - Allegro assai', 2), ('Overdose', 'Deuces Are Wild', 2), ('Not The Doctor', 'Por Causa De Você', 2

{'query': 'Top 10 pair of tracks that were purchased by the same customer. Rank based on the total number of customers who purchased.',
 'result': 'The top 10 pairs of tracks that were purchased by the same customer, ranked based on the total number of customers who purchased, are as follows:\n\n1. Track1: "Snowballed", Track2: "String Quartet No. 12 in C Minor, D. 703 "Quartettsatz": II. Andante - Allegro assai", CustomerCount: 2\n2. Track1: "Overdose", Track2: "Deuces Are Wild", CustomerCount: 2\n3. Track1: "Not The Doctor", Track2: "Por Causa De Você", CustomerCount: 2\n4. Track1: "Not The Doctor", Track2: "Welcome Home (Sanitarium)", CustomerCount: 2\n5. Track1: "Por Causa De Você", Track2: "Welcome Home (Sanitarium)", CustomerCount: 2\n6. Track1: "Samba De Orly", Track2: "Geni E O Zepelim", CustomerCount: 2\n7. Track1: "Samba Do Lado", Track2: "Amor De Muito", CustomerCount: 2\n8. Track1: "Lixo Do Mangue", Track2: "A Cor Do Sol", CustomerCount: 2\n9. Track1: "Lixo Do Mangue", Trac

In [ ]:
db_chain("Which artist on an average has the shortest video tracks in minutes?")



> Entering new SQLDatabaseChain chain...
Which artist on an average has the shortest video tracks in minutes?
SQLQuery:WITH VideoTracks AS (
    SELECT
        t.TrackId,
        t.Name,
        t.Milliseconds / 60000.0 AS DurationInMinutes,
        a.ArtistId,
        a.Name AS ArtistName
    FROM
        Track t
    JOIN
        MediaType mt ON t.MediaTypeId = mt.MediaTypeId
    JOIN
        Genre g ON t.GenreId = g.GenreId
    JOIN
        Album al ON t.AlbumId = al.AlbumId
    JOIN
        Artist a ON al.ArtistId = a.ArtistId
    WHERE
        mt.Name LIKE '%video%'
)
SELECT
    ArtistName,
    AVG(DurationInMinutes) AS AverageDurationInMinutes
FROM
    VideoTracks
GROUP BY
    ArtistId
ORDER BY
    AverageDurationInMinutes ASC;
SQLResult: [('Audioslave', 4.9049), ('The Office', 23.56241037735849), ('Aquaman', 41.40945), ('Lost', 43.1664097826087), ('Heroes', 43.319034782608696), ('Battlestar Galactica', 46.174409166666656), ('Battlestar Galactica (Classic)', 48.75957222222221)]


{'query': 'Which artist on an average has the shortest video tracks in minutes?',
 'result': 'The artist on average with the shortest video tracks in minutes is Audioslave.'}

In [ ]:
db_chain("Rank top 10 countries by video sales revenue. Who is the best video sales rep for customers outside the USA?")



> Entering new SQLDatabaseChain chain...
Rank top 10 countries by video sales revenue. Who is the best video sales rep for customers outside the USA?
SQLQuery:WITH VideoSales AS (
    SELECT
        c.Country,
        SUM(il.UnitPrice * il.Quantity) AS Revenue,
        e.FirstName || ' ' || e.LastName AS SalesRep
    FROM
        Invoice i
    JOIN
        Customer c ON i.CustomerId = c.CustomerId
    JOIN
        InvoiceLine il ON i.InvoiceId = il.InvoiceId
    JOIN
        Track t ON il.TrackId = t.TrackId
    JOIN
        MediaType mt ON t.MediaTypeId = mt.MediaTypeId
    JOIN
        Employee e ON c.SupportRepId = e.EmployeeId
    WHERE
        mt.Name LIKE '%video%'
    GROUP BY
        c.Country, SalesRep
)
SELECT
    Country,
    SUM(Revenue) AS TotalRevenue,
    MAX(SalesRep) AS BestSalesRep
FROM
    VideoSales
WHERE
    Country != 'USA'
GROUP BY
    Country
ORDER BY
    TotalRevenue DESC
LIMIT 10;
SQLResult: [('Czech Republic', 29.849999999999994, 'Steve Johnson'), ('Chile',

{'query': 'Rank top 10 countries by video sales revenue. Who is the best video sales rep for customers outside the USA?',
 'result': 'The top 10 countries ranked by video sales revenue are as follows:\n1. Czech Republic - $29.85\n2. Chile - $17.91\n3. Ireland - $15.92\n4. Hungary - $15.92\n5. France - $13.93\n6. Germany - $11.94\n7. Austria - $9.95\n8. Finland - $7.96\n9. Netherlands - $5.97\n10. Canada - $5.97\n\nThe best video sales representative for customers outside the USA is Steve Johnson.'}

In [ ]:
db_chain("Which pair of artists are most purchased together on the same date? Display the top 10 artists and their purchase frequency.")



> Entering new SQLDatabaseChain chain...
Which pair of artists are most purchased together on the same date? Display the top 10 artists and their purchase frequency.
SQLQuery:WITH ArtistPurchaseDates AS (
    SELECT i.InvoiceDate, a.ArtistId, COUNT(*) as Count
    FROM Invoice i
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    JOIN Track t ON il.TrackId = t.TrackId
    JOIN Album a ON t.AlbumId = a.AlbumId
    GROUP BY i.InvoiceDate, a.ArtistId
)

SELECT ar1.Name as Artist1, ar2.Name as Artist2, SUM(apd1.Count + apd2.Count) as TotalCount
FROM ArtistPurchaseDates apd1
JOIN ArtistPurchaseDates apd2 ON apd1.InvoiceDate = apd2.InvoiceDate AND apd1.ArtistId < apd2.ArtistId
JOIN Artist ar1 ON apd1.ArtistId = ar1.ArtistId
JOIN Artist ar2 ON apd2.ArtistId = ar2.ArtistId
GROUP BY apd1.ArtistId, apd2.ArtistId
ORDER BY TotalCount DESC
LIMIT 10;
SQLResult: [('Lost', 'U2', 48), ('Deep Purple', 'Def Leppard', 35), ('Deep Purple', 'Dennis Chambers', 32), ('Lenny Kravitz', 'Pearl Jam', 31),

{'query': 'Which pair of artists are most purchased together on the same date? Display the top 10 artists and their purchase frequency.',
 'result': "The pair of artists that are most purchased together on the same date are 'Lost' and 'U2', with a purchase frequency of 48."}

In [ ]:
db_chain("Rank the top 5 customer-sales rep pair by total sales.")



> Entering new SQLDatabaseChain chain...
Rank the top 5 customer-sales rep pair by total sales.
SQLQuery:WITH CustomerSales AS (
    SELECT
        c.CustomerId,
        c.FirstName || ' ' || c.LastName AS CustomerName,
        e.EmployeeId,
        e.FirstName || ' ' || e.LastName AS SalesRepName,
        SUM(i.Total) AS TotalSales
    FROM
        Customer c
    JOIN
        Employee e ON c.SupportRepId = e.EmployeeId
    JOIN
        Invoice i ON c.CustomerId = i.CustomerId
    GROUP BY
        c.CustomerId, e.EmployeeId
)
SELECT
    CustomerName,
    SalesRepName,
    TotalSales
FROM
    CustomerSales
ORDER BY
    TotalSales DESC
LIMIT 5;
SQLResult: [('Helena Holý', 'Steve Johnson', 49.620000000000005), ('Richard Cunningham', 'Margaret Park', 47.620000000000005), ('Luis Rojas', 'Steve Johnson', 46.62), ('Ladislav Kovács', 'Jane Peacock', 45.62), ("Hugh O'Reilly", 'Jane Peacock', 45.62)]
Answer:The top 5 customer-sales rep pairs by total sales are:

1. Customer: Helena Holý, Sales

{'query': 'Rank the top 5 customer-sales rep pair by total sales.',
 'result': "The top 5 customer-sales rep pairs by total sales are:\n\n1. Customer: Helena Holý, Sales Rep: Steve Johnson, Total Sales: $49.62\n2. Customer: Richard Cunningham, Sales Rep: Margaret Park, Total Sales: $47.62\n3. Customer: Luis Rojas, Sales Rep: Steve Johnson, Total Sales: $46.62\n4. Customer: Ladislav Kovács, Sales Rep: Jane Peacock, Total Sales: $45.62\n5. Customer: Hugh O'Reilly, Sales Rep: Jane Peacock, Total Sales: $45.62"}

In [ ]:
db_chain("Are my sales going up or down month over month?")



> Entering new SQLDatabaseChain chain...
Are my sales going up or down month over month?
SQLQuery:WITH MonthlySales AS (
    SELECT 
        strftime('%Y-%m', InvoiceDate) AS Month,
        SUM(Total) AS TotalSales
    FROM 
        Invoice
    GROUP BY 
        Month
    ORDER BY 
        Month
)
SELECT 
    Month,
    TotalSales,
    LAG(TotalSales) OVER (ORDER BY Month) AS PreviousMonthSales,
    CASE 
        WHEN TotalSales > LAG(TotalSales) OVER (ORDER BY Month) THEN 'Up'
        WHEN TotalSales < LAG(TotalSales) OVER (ORDER BY Month) THEN 'Down'
        ELSE 'No Change'
    END AS SalesTrend
FROM 
    MonthlySales;
SQLResult: [('2009-01', 35.64, None, 'No Change'), ('2009-02', 37.62, 35.64, 'Up'), ('2009-03', 37.62, 37.62, 'No Change'), ('2009-04', 37.62, 37.62, 'No Change'), ('2009-05', 37.62, 37.62, 'No Change'), ('2009-06', 37.62, 37.62, 'No Change'), ('2009-07', 37.62, 37.62, 'No Change'), ('2009-08', 37.62, 37.62, 'No Change'), ('2009-09', 37.62, 37.62, 'No Change'), ('20

{'query': 'Are my sales going up or down month over month?',
 'history': 'Human: List all distinct playlists and whether they are audio or video.\nAI: There are 15 distinct playlists in the database. Some playlists are labeled as "Audio" while others are labeled as "Video". The "Music" playlist contains both audio and video tracks. The "TV Shows" playlist contains only video tracks. The rest of the playlists are labeled as "Audio".\nHuman: Show me total revenue by country.\nAI: Total revenue by country:\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n11. Ireland: $45.62\n12. Hungary: $45.62\n13. Austria: $42.62\n14. Finland: $41.62\n15. Netherlands: $40.62\n16. Norway: $39.62\n17. Sweden: $38.62\n18. Poland: $37.62\n19. Italy: $37.62\n20. Denmark: $37.62\n21. Australia: $37.62\n22. Argentina: $37.62\n23. Spain: $37.62\n24. B